---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [168]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [169]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [170]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    univ_towns_df = pd.read_fwf('university_towns.txt', header=None)
    univ_towns_df_rows = univ_towns_df.shape[0]
    
    #take a new dataframe to add state and regionname columns and corresponding values
    univ_st_towns_df = pd.DataFrame(columns = ['State', 'RegionName'])
    
    # for every row check whethr edit is in the name and assign to state to if yes
    for i in range(univ_towns_df_rows):
        if 'edit' in univ_towns_df.iloc[i,0]:
            univ_st_towns_df.loc[i,'State'] = univ_towns_df.iloc[i,0]
        univ_st_towns_df.loc[i,'RegionName'] = univ_towns_df.iloc[i,0]
    
    # ffill stname so that it is copied across    
    univ_st_towns_df = univ_st_towns_df.ffill().loc[univ_st_towns_df['State'] != univ_st_towns_df['RegionName']]
             
    # For "State", removing characters from "[" to the end.
    univ_st_towns_df['State'] = univ_st_towns_df['State'].apply(lambda x: x.split('[')[0])
    univ_st_towns_df['RegionName'] = univ_st_towns_df['RegionName'].apply(lambda x: x.split('(')[0].strip() if '(' in x else x)
       
    univ_st_towns_df.reset_index(inplace=True, drop=True)
    
    return univ_st_towns_df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [171]:
def clean_gdp():
    gdp = pd.read_excel('gdplev.xls', header=5)
    
    # use the chained value in 2009 dollars
    gdp_req = gdp.loc[:,['Unnamed: 3', 'GDP in billions of chained 2009 dollars.1']]
    gdp_req = gdp_req.rename(columns={'Unnamed: 3': 'year_qtr'}).set_index('year_qtr')
    
    # only look at GDP data from the first quarter of 2000 onward
    gdp_req = gdp_req.loc['2000q1':, :].dropna()
    return gdp_req
    

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_for_rec = clean_gdp()
    gdp_for_rec['gdp_change'] = gdp_for_rec.diff()
        
    gdp_for_rec['gdp_change_pattern'] = (gdp_for_rec['gdp_change'].gt(gdp_for_rec['gdp_change'].shift(1)))

    # two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth
    pattern = [False, False, True, True]
    
    matched_index =  [gdp_for_rec.index[i - len(pattern)] # Get the datetime index 
     for i in range(len(pattern), len(gdp_for_rec)) # For each 3 consequent elements 
     if all(gdp_for_rec['gdp_change_pattern'][i-len(pattern):i] == pattern)] # If the pattern matched 
    
    # check if there is a way to only identify 2008 recession
    return matched_index[-1]

get_recession_start()


'2008q3'

In [178]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = clean_gdp()
    rec_start = get_recession_start()
    gdp.reset_index(inplace=True)
    rec_stop_idx = gdp[gdp['year_qtr'] == rec_start].index.values[0] + 5

    return gdp.loc[rec_stop_idx, 'year_qtr']

get_recession_end()

'2009q4'

In [173]:
def get_rec_gdp():
    gdp = clean_gdp()
    rec_start = get_recession_start()
    
    gdp.reset_index(inplace=True)
    rec_start_idx = gdp[gdp['year_qtr'] == rec_start].index.values[0]
    
    gdp_rec= gdp.loc[rec_start_idx:rec_start_idx + 3, :]
    
    return gdp_rec


def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp_rec = get_rec_gdp()
    rec_bottom = gdp_rec[['GDP in billions of chained 2009 dollars.1']].idxmin().values[0]
    
    return gdp_rec.loc[rec_bottom, 'year_qtr']

get_recession_bottom()

'2009q2'

In [179]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing['State'] = housing['State'].map(states)
    
    housing = housing.drop(housing.loc[:, '1996-04':'1999-12'], axis=1).set_index(['State', 'RegionName'])
    
    years = np.arange(2000,2017).astype(str)
    
    qtrs = ['q1', 'q2', 'q3', 'q4']
    
    housing_yrs = list(housing.loc[:,'2000-01': '2016-08'])
    i = 0
    j = 2
    
    housing_yrs_len = len(housing_yrs)
    
    for year in years:
        
        for qtr in qtrs:
            if(j < housing_yrs_len):
                housing[year+qtr] = housing.loc[:, housing_yrs[i]:housing_yrs[j]].mean(axis=1)
            i = j+1

            if (year == '2016') and (qtr == 'q2'):
                j = j+2
            else:
                j = j+3
            
    housing_req = housing.drop(housing.loc[:, 'RegionID':'2016-08'], axis=1).sort_index()
    
    return housing_req

convert_housing_data_to_quarters()

2000q1         2000q2         2000q3         2000q4         2001q1         2001q2         2001q3         2001q4         2002q1         2002q2         2002q3        2002q4        2003q1        2003q2        2003q3        2003q4        2004q1        2004q2        2004q3        2004q4        2005q1        2005q2        2005q3        2005q4        2006q1        2006q2        2006q3        2006q4        2007q1        2007q2        2007q3        2007q4        2008q1        2008q2        2008q3        2008q4        2009q1        2009q2        2009q3        2009q4        2010q1        2010q2        2010q3        2010q4        2011q1        2011q2        2011q3        2011q4        2012q1        2012q2        2012q3        2012q4        2013q1        2013q2        2013q3        2013q4        2014q1        2014q2        2014q3        2014q4        2015q1        2015q2        2015q3        2015q4        2016q1        2016q2     2016q3
State     RegionName                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
Alabama   Adamsville                69033.333333   69166.666667   69800.000000   71966.666667   73466.666667   74000.000000   73333.333333   73100.000000   73333.333333   73133.333333   73333.333333  7.573333e+04  7.736667e+04  7.923333e+04  7.976667e+04  7.920000e+04  7.836667e+04  7.696667e+04  7.816667e+04  7.850000e+04  8.030000e+04  8.353333e+04  8.523333e+04  8.580000e+04  8.630000e+04  8.760000e+04  8.783333e+04  8.813333e+04  8.806667e+04  8.750000e+04  8.800000e+04  8.713333e+04  8.583333e+04  8.606667e+04  8.486667e+04  8.583333e+04  8.543333e+04  8.576667e+04  8.476667e+04  8.293333e+04  8.516667e+04  8.263333e+04  8.426667e+04  8.523333e+04  8.560000e+04  8.430000e+04  8.143333e+04  8.150000e+04  7.900000e+04  7.673333e+04  7.633333e+04  7.643333e+04  7.553333e+04  7.816667e+04  7.953333e+04  7.970000e+04  7.856667e+04  7.706667e+04  7.596667e+04  7.190000e+04  7.166667e+04  7.303333e+04  7.393333e+04  7.386667e+04  7.416667e+04  7.493333e+04    74700.0
          Alabaster                122133.333333  123066.666667  123166.666667  123700.000000  123233.333333  125133.333333  127766.666667  127200.000000  127300.000000  128000.000000  127666.666667  1.291000e+05  1.311667e+05  1.341333e+05  1.358000e+05  1.362000e+05  1.366000e+05  1.368667e+05  1.398333e+05  1.448333e+05  1.471333e+05  1.478667e+05  1.491333e+05  1.513667e+05  1.548000e+05  1.570000e+05  1.570333e+05  1.560000e+05  1.565333e+05  1.569000e+05  1.570333e+05  1.573000e+05  1.572667e+05  1.569333e+05  1.553333e+05  1.532000e+05  1.505333e+05  1.496667e+05  1.496000e+05  1.492667e+05  1.478667e+05  1.467667e+05  1.459000e+05  1.439667e+05  1.424333e+05  1.409667e+05  1.402000e+05  1.400000e+05  1.398333e+05  1.395333e+05  1.395000e+05  1.393667e+05  1.395667e+05  1.412333e+05  1.430000e+05  1.448000e+05  1.466333e+05  1.471333e+05  1.476333e+05  1.487000e+05  1.489000e+05  1.495667e+05  1.503667e+05  1.517333e+05  1.534667e+05  1.551000e+05   155850.0
          Albertville               73966.666667   72600.000000   72833.333333   74200.000000   75900.000000   76000.00000

In [195]:
from scipy.stats import ttest_ind

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    univ = get_list_of_university_towns()
    
    gdp_rec = get_rec_gdp()
    
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    
    housing = convert_housing_data_to_quarters().reset_index().sort_values(['State', 'RegionName'])

    housing['price_change'] = housing[rec_start] - housing[rec_bottom]

    bstart = housing.columns[housing.columns.get_loc(rec_start) -1]
    housing['price_ratio'] = housing[rec_bottom] - housing[rec_start]

    # get univ towns housing info
    univ_housing = housing.merge(univ, on=['State', 'RegionName'])
    
    # get non-univ towns housing info
    non_univ_housing = pd.concat([housing, univ_housing]).loc[
    housing.index.symmetric_difference(univ_housing.index)]
    
    t,p = ttest_ind(univ_housing['price_ratio'].dropna(), non_univ_housing['price_ratio'].dropna())
    
    different = True if p < 0.01 else False
    
    better = 'non-university town' if univ_housing['price_ratio'].mean() < non_univ_housing['price_ratio'].mean() else 'university town'
    
    return different, p, better

run_ttest()


(True, 0.0052917058569850912, 'university town')